In [37]:
import json
import numpy as np

with open("../data/covid19-cuba.json") as fp:
    factors = json.load(fp)['casos']['dias']
    cumul = {
        int(k): d.get('tests_total', 0) for k,d in factors.items()
    }
    
    
#     factors = {int(k): v for k,v in factors.items()}
#     cumul = {1: factors[1]}
    
#     for i in range(2, 12):
#         cumul[i] = cumul[i-1] + factors.get(i, 0)

#     extra = [518, 625, 738, 893, 1039, 1368, 1665, 2007, 2322]
    
#     for i in range(12, 21):
#         cumul[i] = extra[i - 12]
                   
cumul[26] = cumul[25] + 764
cumul


{1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 518,
 13: 625,
 14: 738,
 15: 893,
 16: 1039,
 17: 1368,
 18: 1665,
 19: 2007,
 20: 2322,
 21: 2766,
 22: 3342,
 23: 4145,
 24: 5202,
 25: 6403,
 26: 7167}

In [35]:
with open("../data/timeseries.json") as fp:
    cases = [d['confirmed'] for d in json.load(fp)['Cuba'] if d['confirmed'] > 0]
    cases = {i+1: d for i,d in enumerate(cases)}
    
cases[20] = 212
    
# alpha que mejor aproxima los datos (error cuadrático medio)
# alpha = []

# for i in range(2, 19):
#     alpha.append(cases[i] / cases[i-1])
    
# alpha = sum(alpha) / len(alpha)

alpha = 1.111 # alpha del modelo aprendido en el notebook `predict_cuba.ipynb` que supuestamente está mejor (generaliza mejor 🙏)
alpha

1.111

In [36]:
cases

{1: 3,
 2: 4,
 3: 4,
 4: 4,
 5: 4,
 6: 5,
 7: 7,
 8: 11,
 9: 16,
 10: 21,
 11: 35,
 12: 40,
 13: 48,
 14: 57,
 15: 67,
 16: 80,
 17: 119,
 18: 139,
 19: 170,
 20: 212,
 21: 212,
 22: 233,
 23: 269,
 24: 288,
 25: 320,
 26: 350}

In [22]:
predicted = {1: cases[1]}

for i in range(2, 27):
    predicted[i] = round(cases[i-1] * alpha)
    
predicted

{1: 3,
 2: 3,
 3: 4,
 4: 4,
 5: 4,
 6: 4,
 7: 6,
 8: 8,
 9: 12,
 10: 18,
 11: 23,
 12: 39,
 13: 44,
 14: 53,
 15: 63,
 16: 74,
 17: 89,
 18: 132,
 19: 154,
 20: 189,
 21: 236,
 22: 236,
 23: 259,
 24: 299,
 25: 320,
 26: 356}

In [23]:
def error(t1, t2, exponent=1, normalize=True):
    def metric(vi, vj):
        t = abs(vi - vj)
        b = abs(vi) if normalize else 1
        return (t / b) ** exponent
    
    residuals = [metric(vi, vj) for vi,vj in zip(t1, t2)]
    msqe = sum(residuals) / len(residuals)
    
    return msqe

In [53]:
import pandas as pd

paired = pd.DataFrame([dict(day=i, cases=cases[i], factor=cumul[i], new=cumul[i] - cumul.get(i-1,cumul[i]), percent=cases[i]/(cumul[i]+1), predicted=predicted[i]) for i in cases if i >= 0])
paired['errors'] = paired['cases'] - paired['predicted']
paired

,day,cases,factor,new,percent,predicted,errors
0,1,3,0,0,3.000000,3,0
1,2,4,0,0,4.000000,3,1
2,3,4,0,0,4.000000,4,0
3,4,4,0,0,4.000000,4,0
4,5,4,0,0,4.000000,4,0
5,6,5,0,0,5.000000,4,1
6,7,7,0,0,7.000000,6,1
7,8,11,0,0,11.000000,8,3
8,9,16,0,0,16.000000,12,4
9,10,21,0,0,21.000000,18,3


In [54]:
error(paired['cases'][-7:], paired['predicted'][-7:])

0.04672652463787522

In [55]:
error(paired['cases'], paired['predicted'])

0.10146818607138999

In [56]:
import altair as alt

alt.Chart(paired).mark_line().encode(
    x='day',
    y='new'
) + alt.Chart(paired).mark_line(color="red").encode(
    x='day',
    y='errors'
).properties(width=300, height=500)

alt.LayerChart(...)

In [58]:
alt.Chart(paired).mark_line(color='green').encode(
    x='day',
    y='cases'
) + alt.Chart(paired).mark_line(color='orange').encode(
    x='day',
    y='predicted'
) + alt.Chart(paired).mark_bar(color='red', width=2).encode(
    x='day',
    y='cases',
    y2='predicted'
).properties(width=400, height=500)

alt.LayerChart(...)

In [42]:
X = paired['new'].values
y = paired['errors'].values

In [43]:
from sklearn.linear_model import LinearRegression

Xt = X.reshape(-1,1)

model = LinearRegression()
model.fit(Xt, y)
model.score(Xt, y)

0.12374966556776035

In [44]:
paired['fix'] = model.predict(Xt).astype(int)
paired['predicted+fixed'] = paired['predicted'] + paired['fix']
paired

,day,cases,factor,new,percent,predicted,errors,fix,predicted+fixed
0,12,40,518,518,0.077220,39,1,3,42
1,13,48,625,107,0.076800,44,4,9,53
2,14,57,738,113,0.077236,53,4,8,61
3,15,67,893,155,0.075028,63,4,8,71
4,16,80,1039,146,0.076997,74,6,8,82
5,17,119,1368,329,0.086988,89,30,6,95
6,18,139,1665,297,0.083483,132,7,6,138
7,19,170,2007,342,0.084704,154,16,5,159
8,20,212,2322,315,0.091301,189,23,6,195
9,21,212,2766,444,0.076645,236,-24,4,240


In [45]:
error_before = error(paired['cases'], paired['predicted'])
error_after = error(paired['cases'], paired['predicted+fixed'])

error_before, error_after

(0.06912494243716255, 0.060937875436537996)

In [50]:
alt.Chart(paired).mark_line(color='green').encode(
    x='day',
    y='cases'
) + alt.Chart(paired).mark_line(color='orange').encode(
    x='day',
    y='predicted+fixed'
) + alt.Chart(paired).mark_bar(color='red', width=2).encode(
    x='day',
    y='cases',
    y2='predicted+fixed'
).properties(width=300, height=500)

alt.LayerChart(...)